In [14]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import random
import pickle
import pandas as pd
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import roc_curve

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import imblearn
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report

In [2]:
os.chdir("/homes/t326h379/OGalNAc/Ankh_154_Proteins")

df_positive = pd.read_csv("Again_Ankh_Feature_Extraction_667_Positive_OGalNAc.txt",header=None)
df_negative = pd.read_csv("Again_Ankh_Feature_Extraction_17817_Negative_OGalNAc.txt",header = None)

df_positive_positive = df_positive
df_negative_negative = df_negative

Header_name = ["Position","PID","S_or_T"]+[i for i in range(1,1537)]
df_positive_positive.columns = Header_name
df_negative_negative.columns = Header_name

Training_Protein = []
with open("One_hundred_Thirty_Seven_Training_Proteins_OGalNAc.txt") as Subash:
    for line in Subash:
        Training_Protein.append(line.strip("\n"))

df_positive = df_positive_positive[df_positive_positive['PID'].isin(Training_Protein)]
df_negative = df_negative_negative[df_negative_negative['PID'].isin(Training_Protein)]

In [4]:
List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
len(List_of_Training_Protein)

137

# MLP 10 Fold CV

In [15]:
from imblearn.under_sampling import RandomUnderSampler
a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        model = Sequential()

        model.add(Input(shape=(1536,)))
        
        model.add(Dense(512,activation='relu'))
        model.add(Dropout(0.3))
        
        model.add(Dense(256,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(32,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))



        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train, y_train_1, epochs=12,verbose=1, batch_size=32)

        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(530, 1536) (38, 1536)
(13746, 1536) (1464, 1536)
Training Dataset
(1060, 1536) (1060,)
Validation Dataset
(76, 1536) (76,)




Epoch 1/12
34/34 [==============================] - 1s 4ms/step - loss: 0.6478 - accuracy: 0.6396
Epoch 2/12
34/34 [==============================] - 0s 4ms/step - loss: 0.4838 - accuracy: 0.7934
Epoch 3/12
34/34 [==============================] - 0s 4ms/step - loss: 0.3884 - accuracy: 0.8340
Epoch 4/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2804 - accuracy: 0.8962
Epoch 5/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2620 - accuracy: 0.9019
Epoch 6/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1989 - accuracy: 0.9321
Epoch 7/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1352 - accuracy: 0.9500
Epoch 8/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1353 - accuracy: 0.9491
Epoch 9/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1381 - accurac

(492, 1536) (76, 1536)
(13787, 1536) (1423, 1536)
Training Dataset
(984, 1536) (984,)
Validation Dataset
(152, 1536) (152,)




Epoch 1/12
31/31 [==============================] - 1s 4ms/step - loss: 0.6595 - accuracy: 0.6270
Epoch 2/12
31/31 [==============================] - 0s 4ms/step - loss: 0.4968 - accuracy: 0.7876
Epoch 3/12
31/31 [==============================] - 0s 4ms/step - loss: 0.3850 - accuracy: 0.8384
Epoch 4/12
31/31 [==============================] - 0s 4ms/step - loss: 0.2874 - accuracy: 0.8862
Epoch 5/12
31/31 [==============================] - 0s 4ms/step - loss: 0.2255 - accuracy: 0.9126
Epoch 6/12
31/31 [==============================] - 0s 4ms/step - loss: 0.1938 - accuracy: 0.9258
Epoch 7/12
31/31 [==============================] - 0s 4ms/step - loss: 0.1279 - accuracy: 0.9614
Epoch 8/12
31/31 [==============================] - 0s 4ms/step - loss: 0.1270 - accuracy: 0.9512
Epoch 9/12
31/31 [==============================] - 0s 4ms/step - loss: 0.0964 - accurac

2/2 [==============================] - 0s 2ms/step
Independent Testing Matthews Correlation:  0.3363363969981562
Confusion Matrix : 
 [[22  8]
 [12 18]]
Accuracy on test set:    0.6666666666666666
Sensitivity:    0.6 	 Specificity:    0.7333333333333333
Precision  : 0.6923076923076923
Area Under Curve:    0.6666666666666667
(60, 1536) (60,)
(520, 1536) (58, 1536)
(14082, 1536) (1478, 1536)
Training Dataset
(1040, 1536) (1040,)
Validation Dataset
(116, 1536) (116,)




Epoch 1/12
33/33 [==============================] - 1s 4ms/step - loss: 0.6534 - accuracy: 0.6413
Epoch 2/12
33/33 [==============================] - 0s 4ms/step - loss: 0.4847 - accuracy: 0.7894
Epoch 3/12
33/33 [==============================] - 0s 4ms/step - loss: 0.3692 - accuracy: 0.8462
Epoch 4/12
33/33 [==============================] - 0s 4ms/step - loss: 0.3001 - accuracy: 0.8875
Epoch 5/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2481 - accuracy: 0.9144
Epoch 6/12
33/33 [===================

33/33 [==============================] - 0s 4ms/step - loss: 0.2582 - accuracy: 0.9114
Epoch 6/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2160 - accuracy: 0.9267
Epoch 7/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2051 - accuracy: 0.9171
Epoch 8/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1478 - accuracy: 0.9533
Epoch 9/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1294 - accuracy: 0.9581
Epoch 10/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1130 - accuracy: 0.9610
Epoch 11/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1135 - accuracy: 0.9514
Epoch 12/12
4/4 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.6471789394828489
Confusion Matrix : 
 [[47  6]
 [13 40]]
Accuracy on test set:    0.8207547169811321
Sensitivity:    0.7547169811320755 	 Specificity:    0.8867924528301887
Precision  : 0.8695652173913043
Area Under

33/33 [==============================] - 1s 4ms/step - loss: 0.6227 - accuracy: 0.6893
Epoch 2/12
33/33 [==============================] - 0s 4ms/step - loss: 0.4515 - accuracy: 0.8088
Epoch 3/12
33/33 [==============================] - 0s 4ms/step - loss: 0.3579 - accuracy: 0.8623
Epoch 4/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2896 - accuracy: 0.8881
Epoch 5/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2434 - accuracy: 0.9073
Epoch 6/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2193 - accuracy: 0.9187
Epoch 7/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1473 - accuracy: 0.9493
Epoch 8/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1441 - accuracy: 0.9493
Epoch 9/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1055 - accuracy: 0.9589
Epoch 10/12
33/33 [==============================] - 0s 4ms/step - loss: 0.0895 - accuracy: 0.9646
Epoch 11/12
33/33 [===========

...

33/33 [==============================] - 0s 4ms/step - loss: 0.3170 - accuracy: 0.8745
Epoch 4/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2504 - accuracy: 0.9102
Epoch 5/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2197 - accuracy: 0.9170
Epoch 6/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1470 - accuracy: 0.9527
Epoch 7/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1176 - accuracy: 0.9604
Epoch 8/12
33/33 [==============================] - 0s 4ms/step - loss: 0.0957 - accuracy: 0.9749
Epoch 9/12
33/33 [==============================] - 0s 4ms/step - loss: 0.0636 - accuracy: 0.9768
Epoch 10/12
33/33 [==============================] - 0s 4ms/step - loss: 0.0462 - accuracy: 0.9846
Epoch 11/12
33/33 [==============================] - 0s 4ms/step - loss: 0.0444 - accuracy: 0.9855
Epoch 12/12
4/4 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.4601950115131127
Co

33/33 [==============================] - 2s 4ms/step - loss: 0.6264 - accuracy: 0.6755
Epoch 2/12
33/33 [==============================] - 0s 4ms/step - loss: 0.4548 - accuracy: 0.8112
Epoch 3/12
33/33 [==============================] - 0s 4ms/step - loss: 0.3688 - accuracy: 0.8482
Epoch 4/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2755 - accuracy: 0.9013
Epoch 5/12
33/33 [==============================] - 0s 4ms/step - loss: 0.2263 - accuracy: 0.9260
Epoch 6/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1948 - accuracy: 0.9364
Epoch 7/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1728 - accuracy: 0.9355
Epoch 8/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1433 - accuracy: 0.9478
Epoch 9/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1420 - accuracy: 0.9507
Epoch 10/12
33/33 [==============================] - 0s 4ms/step - loss: 0.1240 - accuracy: 0.9535
Epoch 11/12
33/33 [===========

(504, 1536) (69, 1536)
(14293, 1536) (1335, 1536)
Training Dataset
(1008, 1536) (1008,)
Validation Dataset
(138, 1536) (138,)




Epoch 1/12
32/32 [==============================] - 1s 4ms/step - loss: 0.6522 - accuracy: 0.6538
Epoch 2/12
32/32 [==============================] - 0s 4ms/step - loss: 0.4921 - accuracy: 0.7748
Epoch 3/12
32/32 [==============================] - 0s 4ms/step - loss: 0.3769 - accuracy: 0.8502
Epoch 4/12
32/32 [==============================] - 0s 4ms/step - loss: 0.2905 - accuracy: 0.8919
Epoch 5/12
32/32 [==============================] - 0s 4ms/step - loss: 0.2361 - accuracy: 0.9196
Epoch 6/12
32/32 [==============================] - 0s 4ms/step - loss: 0.1907 - accuracy: 0.9375
Epoch 7/12
32/32 [==============================] - 0s 4ms/step - loss: 0.1227 - accuracy: 0.9623
Epoch 8/12
32/32 [==============================] - 0s 4ms/step - loss: 0.1218 - accuracy: 0.9573
Epoch 9/12
32/32 [==============================] - 0s 4ms/step - loss: 0.1070 - accur

(529, 1536) (49, 1536)
(14386, 1536) (1331, 1536)
Training Dataset
(1058, 1536) (1058,)
Validation Dataset
(98, 1536) (98,)




Epoch 1/12
34/34 [==============================] - 1s 4ms/step - loss: 0.6538 - accuracy: 0.6427
Epoch 2/12
34/34 [==============================] - 0s 4ms/step - loss: 0.4954 - accuracy: 0.8062
Epoch 3/12
34/34 [==============================] - 0s 4ms/step - loss: 0.3940 - accuracy: 0.8488
Epoch 4/12
34/34 [==============================] - 0s 4ms/step - loss: 0.3172 - accuracy: 0.8771
Epoch 5/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2650 - accuracy: 0.9130
Epoch 6/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2039 - accuracy: 0.9197
Epoch 7/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2121 - accuracy: 0.9282
Epoch 8/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1609 - accuracy: 0.9310
Epoch 9/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1464 - accurac

(534, 1536) (44, 1536)
(14454, 1536) (1263, 1536)
Training Dataset
(1068, 1536) (1068,)
Validation Dataset
(88, 1536) (88,)




Epoch 1/12
34/34 [==============================] - 2s 4ms/step - loss: 0.6393 - accuracy: 0.6751
Epoch 2/12
34/34 [==============================] - 0s 4ms/step - loss: 0.4595 - accuracy: 0.8015
Epoch 3/12
34/34 [==============================] - 0s 4ms/step - loss: 0.3446 - accuracy: 0.8642
Epoch 4/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2639 - accuracy: 0.9045
Epoch 5/12
34/34 [==============================] - 0s 4ms/step - loss: 0.2047 - accuracy: 0.9326
Epoch 6/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1702 - accuracy: 0.9391
Epoch 7/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1476 - accuracy: 0.9429
Epoch 8/12
34/34 [==============================] - 0s 4ms/step - loss: 0.1166 - accuracy: 0.9569
Epoch 9/12
34/34 [==============================] - 0s 4ms/step - loss: 0.0763 - accurac

In [12]:
print(X_train.shape,y_train.shape)

print("Validation Dataset")
print(xval.shape,yval.shape)

(1076, 1536) (1076,)
Validation Dataset
(72, 1536) (72,)


# Logistic Regression 10 Fold CV

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
a = random.sample(range(1, 1000000), 10)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("LR Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("LR Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(521, 1536) (58, 1536)
(12718, 1536) (3057, 1536)
Training Dataset
(1042, 1536) (1042,)
Validation Dataset
(116, 1536) (116,)




Matthews Correlation :  0.34565056491014173
Confusion Matrix : 
 [[41 17]
 [21 37]]
Accuracy on test set:    0.6724137931034483
Sensitivity:    0.6379310344827587 	 Specificity:    0.7068965517241379
Area Under Curve:    0.6724137931034483
(536, 1536) (43, 1536)
(14282, 1536) (1493, 1536)
Training Dataset
(1072, 1536) (1072,)
Validation Dataset
(86, 1536) (86,)




Matthews Correlation :  0.5354632267566766
Confusion Matrix : 
 [[34  9]
 [11 32]]
Accuracy on test set:    0.7674418604651163
Sensitivity:    0.7441860465116279 	 Specificity:    0.7906976744186046
Area Under Curve:    0.7674418604651162
(534, 1536) (45, 1536)
(14891, 1536) (884, 1536)
Training Dataset
(1068, 1536) (1068,)
Validation Dataset
(90, 1536) (90,)




Matthews Correlation :  0.6502503675297687
Confusion Matrix : 
 [[34 11]
 [ 5 40]]
Accuracy on test set:    0.8222222222222222
Sensitivi

(414, 1536) (165, 1536)
(12620, 1536) (3155, 1536)
Training Dataset
(828, 1536) (828,)
Validation Dataset
(330, 1536) (330,)




Matthews Correlation :  0.35970073030870453
Confusion Matrix : 
 [[129  36]
 [ 71  94]]
Accuracy on test set:    0.6757575757575758
Sensitivity:    0.5696969696969697 	 Specificity:    0.7818181818181819
Area Under Curve:    0.6757575757575758
(516, 1536) (63, 1536)
(14318, 1536) (1457, 1536)
Training Dataset
(1032, 1536) (1032,)
Validation Dataset
(126, 1536) (126,)




Matthews Correlation :  0.4603174603174603
Confusion Matrix : 
 [[46 17]
 [17 46]]
Accuracy on test set:    0.7301587301587301
Sensitivity:    0.7301587301587301 	 Specificity:    0.7301587301587301
Area Under Curve:    0.7301587301587302
(518, 1536) (61, 1536)
(14041, 1536) (1734, 1536)
Training Dataset
(1036, 1536) (1036,)
Validation Dataset
(122, 1536) (122,)




Matthews Correlation :  0.6118419408382955
Confusion Matrix : 
 [[45 16]
 [ 8 53]]
Accuracy on test set:    0.8032786885245902
S

(519, 1536) (59, 1536)
(13888, 1536) (1789, 1536)
Training Dataset
(1038, 1536) (1038,)
Validation Dataset
(118, 1536) (118,)




Matthews Correlation :  0.4578902739956863
Confusion Matrix : 
 [[42 17]
 [15 44]]
Accuracy on test set:    0.7288135593220338
Sensitivity:    0.7457627118644068 	 Specificity:    0.711864406779661
Area Under Curve:    0.728813559322034
LR Mean MCC = 0.4769 ± 0.0999 and Mean AUC = 0.7354 ± 0.0494
LR Mean MCC:    0.47690236959349125
Mean Sensitivity = 0.7352 ± 0.0976 and Mean Specificity = 0.7356 ± 0.0889 and Mean Accuracy = 0.7354 ± 0.0494
Mean Precision = 0.7414 ± 0.0667 














(537, 1536) (36, 1536)
(14347, 1536) (1313, 1536)
Training Dataset
(1074, 1536) (1074,)
Validation Dataset
(72, 1536) (72,)




Matthews Correlation :  0.5835585150955648
Confusion Matrix : 
 [[28  8]
 [ 7 29]]
Accuracy on test set:    0.7916666666666666
Sensitivity:    0.8055555555555556 	 Specificity:    0.7777777777777778
Area Under Curve:    0.7916666666666666
(520, 1536)

(521, 1536) (50, 1536)
(14846, 1536) (949, 1536)
Training Dataset
(1042, 1536) (1042,)
Validation Dataset
(100, 1536) (100,)




Matthews Correlation :  0.5216720300383332
Confusion Matrix : 
 [[36 14]
 [10 40]]
Accuracy on test set:    0.76
Sensitivity:    0.8 	 Specificity:    0.72
Area Under Curve:    0.76
(524, 1536) (47, 1536)
(12357, 1536) (3438, 1536)
Training Dataset
(1048, 1536) (1048,)
Validation Dataset
(94, 1536) (94,)




Matthews Correlation :  0.47519096331149147
Confusion Matrix : 
 [[26 21]
 [ 5 42]]
Accuracy on test set:    0.723404255319149
Sensitivity:    0.8936170212765957 	 Specificity:    0.5531914893617021
Area Under Curve:    0.7234042553191489
(412, 1536) (159, 1536)
(14396, 1536) (1399, 1536)
Training Dataset
(824, 1536) (824,)
Validation Dataset
(318, 1536) (318,)




Matthews Correlation :  0.3471825374147068
Confusion Matrix : 
 [[ 90  69]
 [ 36 123]]
Accuracy on test set:    0.6698113207547169
Sensitivity:    0.7735849056603774 	 Specificity:    0.5660377

(414, 1536) (155, 1536)
(13534, 1536) (1917, 1536)
Training Dataset
(828, 1536) (828,)
Validation Dataset
(310, 1536) (310,)




Matthews Correlation :  0.24174688920761409
Confusion Matrix : 
 [[117  38]
 [ 81  74]]
Accuracy on test set:    0.6161290322580645
Sensitivity:    0.4774193548387097 	 Specificity:    0.7548387096774194
Area Under Curve:    0.6161290322580646
(534, 1536) (35, 1536)
(14385, 1536) (1066, 1536)
Training Dataset
(1068, 1536) (1068,)
Validation Dataset
(70, 1536) (70,)




Matthews Correlation :  0.7431605356175383
Confusion Matrix : 
 [[30  5]
 [ 4 31]]
Accuracy on test set:    0.8714285714285714
Sensitivity:    0.8857142857142857 	 Specificity:    0.8571428571428571
Area Under Curve:    0.8714285714285714
(506, 1536) (63, 1536)
(14099, 1536) (1352, 1536)
Training Dataset
(1012, 1536) (1012,)
Validation Dataset
(126, 1536) (126,)




Matthews Correlation :  0.49231163302332015
Confusion Matrix : 
 [[46 17]
 [15 48]]
Accuracy on test set:    0.746031746031746
Sen

# Random Forest 10 Fold CV

In [7]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
a = random.sample(range(1, 1000000), 4)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = RandomForestClassifier(max_depth=10, random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("RF Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("RF Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(536, 1536) (37, 1536)
(14032, 1536) (1694, 1536)
Training Dataset
(1072, 1536) (1072,)
Validation Dataset
(74, 1536) (74,)




Matthews Correlation :  0.5259547057403531
Confusion Matrix : 
 [[24 13]
 [ 5 32]]
Accuracy on test set:    0.7567567567567568
Sensitivity:    0.8648648648648649 	 Specificity:    0.6486486486486487
Area Under Curve:    0.7567567567567568
(520, 1536) (53, 1536)
(14477, 1536) (1249, 1536)
Training Dataset
(1040, 1536) (1040,)
Validation Dataset
(106, 1536) (106,)




Matthews Correlation :  0.3829197905337418
Confusion Matrix : 
 [[32 21]
 [12 41]]
Accuracy on test set:    0.6886792452830188
Sensitivity:    0.7735849056603774 	 Specificity:    0.6037735849056604
Area Under Curve:    0.6886792452830188
(530, 1536) (43, 1536)
(14274, 1536) (1452, 1536)
Training Dataset
(1060, 1536) (1060,)
Validation Dataset
(86, 1536) (86,)




Matthews Correlation :  0.44378474407369917
Confusion Matrix : 
 [[29 14]
 [10 33]]
Accuracy on test set:    0.7209302325581395
Sensitiv

(526, 1536) (54, 1536)
(14271, 1536) (1409, 1536)
Training Dataset
(1052, 1536) (1052,)
Validation Dataset
(108, 1536) (108,)




Matthews Correlation :  0.476991542824123
Confusion Matrix : 
 [[46  8]
 [21 33]]
Accuracy on test set:    0.7314814814814815
Sensitivity:    0.6111111111111112 	 Specificity:    0.8518518518518519
Area Under Curve:    0.7314814814814815
(521, 1536) (59, 1536)
(14746, 1536) (934, 1536)
Training Dataset
(1042, 1536) (1042,)
Validation Dataset
(118, 1536) (118,)




Matthews Correlation :  0.2942868240199161
Confusion Matrix : 
 [[44 15]
 [27 32]]
Accuracy on test set:    0.6440677966101694
Sensitivity:    0.5423728813559322 	 Specificity:    0.7457627118644068
Area Under Curve:    0.6440677966101696
(533, 1536) (47, 1536)
(13991, 1536) (1689, 1536)
Training Dataset
(1066, 1536) (1066,)
Validation Dataset
(94, 1536) (94,)




Matthews Correlation :  0.5135525910130955
Confusion Matrix : 
 [[33 14]
 [ 9 38]]
Accuracy on test set:    0.7553191489361702
Sensitivi

# XGBoost 10 Fold CV

In [17]:
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 3)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = XGBClassifier()

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("XGB Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("XGB Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(511, 1536) (58, 1536)
(12601, 1536) (1276, 1536)
Training Dataset
(1022, 1536) (1022,)
Validation Dataset
(116, 1536) (116,)




Matthews Correlation :  0.43900203592056264
Confusion Matrix : 
 [[47 11]
 [22 36]]
Accuracy on test set:    0.7155172413793104
Sensitivity:    0.6206896551724138 	 Specificity:    0.8103448275862069
Area Under Curve:    0.7155172413793103
(539, 1536) (30, 1536)
(12219, 1536) (1658, 1536)
Training Dataset
(1078, 1536) (1078,)
Validation Dataset
(60, 1536) (60,)




Matthews Correlation :  0.4394802124588286
Confusion Matrix : 
 [[19 11]
 [ 6 24]]
Accuracy on test set:    0.7166666666666667
Sensitivity:    0.8 	 Specificity:    0.6333333333333333
Area Under Curve:    0.7166666666666666
(425, 1536) (144, 1536)
(12227, 1536) (1650, 1536)
Training Dataset
(850, 1536) (850,)
Validation Dataset
(288, 1536) (288,)




Matthews Correlation :  0.3359684204526464
Confusion Matrix : 
 [[105  39]
 [ 57  87]]
Accuracy on test set:    0.6666666666666666
Sensitivity:    0.

(502, 1536) (63, 1536)
(14696, 1536) (1062, 1536)
Training Dataset
(1004, 1536) (1004,)
Validation Dataset
(126, 1536) (126,)




Matthews Correlation :  0.41400434094401334
Confusion Matrix : 
 [[42 21]
 [16 47]]
Accuracy on test set:    0.7063492063492064
Sensitivity:    0.746031746031746 	 Specificity:    0.6666666666666666
Area Under Curve:    0.7063492063492064
(521, 1536) (44, 1536)
(14416, 1536) (1342, 1536)
Training Dataset
(1042, 1536) (1042,)
Validation Dataset
(88, 1536) (88,)




Matthews Correlation :  0.4090909090909091
Confusion Matrix : 
 [[31 13]
 [13 31]]
Accuracy on test set:    0.7045454545454546
Sensitivity:    0.7045454545454546 	 Specificity:    0.7045454545454546
Area Under Curve:    0.7045454545454546
(513, 1536) (52, 1536)
(13162, 1536) (2596, 1536)
Training Dataset
(1026, 1536) (1026,)
Validation Dataset
(104, 1536) (104,)




Matthews Correlation :  0.4259217009643015
Confusion Matrix : 
 [[34 18]
 [12 40]]
Accuracy on test set:    0.7115384615384616
Sensiti

# SVM 10 Fold CV

In [16]:
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 1)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = svm.SVC(C=10, kernel='rbf')

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("SVM Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("SVM Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(515, 1536) (59, 1536)
(14199, 1536) (1564, 1536)
Training Dataset
(1030, 1536) (1030,)
Validation Dataset
(118, 1536) (118,)




Matthews Correlation :  0.5366406790951412
Confusion Matrix : 
 [[51  8]
 [20 39]]
Accuracy on test set:    0.7627118644067796
Sensitivity:    0.6610169491525424 	 Specificity:    0.864406779661017
Area Under Curve:    0.7627118644067797
(532, 1536) (42, 1536)
(14347, 1536) (1416, 1536)
Training Dataset
(1064, 1536) (1064,)
Validation Dataset
(84, 1536) (84,)




Matthews Correlation :  0.6445033866354896
Confusion Matrix : 
 [[33  9]
 [ 6 36]]
Accuracy on test set:    0.8214285714285714
Sensitivity:    0.8571428571428571 	 Specificity:    0.7857142857142857
Area Under Curve:    0.8214285714285714
(534, 1536) (40, 1536)
(14303, 1536) (1460, 1536)
Training Dataset
(1068, 1536) (1068,)
Validation Dataset
(80, 1536) (80,)




Matthews Correlation :  0.7008766440504625
Confusion Matrix : 
 [[33  7]
 [ 5 35]]
Accuracy on test set:    0.85
Sensitivity:    0.875 	 